# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark SQL: Soluciones de almacenamiento para Big Data (Postgres)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [32]:
from pyspark.sql import SparkSession

# Hint for homework: .config("spark.jars.packages", "groupId:artifactId:version") \

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Postgres-Write") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars", "/home/jovyan/notebooks/jars/postgresql-42.7.5.jar") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

#### Cargamos el DF de Netflix

In [3]:
from team_name.spark_utils import SparkUtils

columns_info = [("show_id", "string"),("type", "string"),("title", "string"),("director", "string"),("country", "string"),("date_added", "string"),("release_year", "integer"),("rating", "string"), ("duration", "string"),("listed_in", "string")]

netflix_schema = SparkUtils.generate_schema(columns_info)

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

#### Escribir un DF a Postgres

In [5]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"

netflix_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "netflix") \
    .option("user", "postgres") \
    .option("password", "Admin@1234") \
    .option("driver", "org.postgresql.Driver") \
    .save()

print("Done")

Done


In [6]:
# Stop the SparkContext
sc.stop()

#### Leer info desde PostgreSQL

In [12]:
spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Postgres-Read") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars", "/home/jovyan/notebooks/jars/postgresql-42.7.5.jar") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

##### Propiedades de conexión con la Base de Datos remota

In [13]:
# Database connection properties
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
db_properties = {
      "user": "postgres",      
      "password": "Admin@1234",
      "driver": "org.postgresql.Driver"
  }

df = spark.read \
    .jdbc(url=jdbc_url, table="netflix", properties=db_properties)

df.printSchema()
df.show(5, False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

In [9]:
sc.stop()

#### Dgraph example

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Dgraph") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-dgraph-connector_2.12:0.12.0-3.5") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [19]:
from dgraph_connector.gresearch.spark.dgraph.connector import *

df = spark \
  .read \
  .option(NodesModeOption, NodesModeWideOption) \
  .dgraph.nodes("dgraph-iteso:9080")

df.printSchema()


root
 |-- subject: long (nullable = false)
 |-- age: long (nullable = true)
 |-- dgraph.drop.op: string (nullable = true)
 |-- dgraph.graphql.p_query: string (nullable = true)
 |-- dgraph.graphql.schema: string (nullable = true)
 |-- dgraph.graphql.xid: string (nullable = true)
 |-- dgraph.type: string (nullable = true)
 |-- email: string (nullable = true)
 |-- name: string (nullable = true)



In [21]:
sc.stop()